# BOFire Reaction Optimization Example

This notebook follows the example set out in https://github.com/experimental-design/bofire/blob/main/tutorials/basic_examples/Reaction_Optimization_Example.ipynb

In [1]:
# python imports we'll need in this notebook
from pprint import pprint as pp
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
import os

## Setting up the optimization problem as a Reaction Domain


In [2]:
from bofire.data_models.domain.api import Domain
from bofire.data_models.domain.api import Inputs, Outputs
from bofire.data_models.features.api import ContinuousInput, ContinuousOutput, CategoricalInput, CategoricalDescriptorInput  # we won't need all of those.

In [3]:
# We wish the temperature of the reaction to be between 30 and 110 °C
temperature_feature = ContinuousInput(key='Temperature', bounds = [30.0, 110.0], unit='°C')

# Catalyst Loading
catalyst_loading_feature = ContinuousInput(key='Catalyst Loading', bounds=[0.5,2], unit="%")

# Residence Time
residence_time_feature = ContinuousInput(key='Residence Time', bounds=[1, 10], unit="minutes")

# Catalyst choice
catalyst_feature = CategoricalInput(
    key = 'Catalyst', 
    categories = ['[P1-L1]','[P2-L1]','[P1-L2]', '[P1-L3]', '[P1-L4]', '[P1-L5]', '[P1-L6]', '[P1-L7]']
)

# gather all individual features
input_features = Inputs(
    features = [
        temperature_feature,
        catalyst_loading_feature,
        residence_time_feature, 
        catalyst_feature,
    ]
)

In [4]:
# outputs: we wish to maximize the Yield
# import Maximize Objective to tell the optimizer you wish to optimize
from bofire.data_models.objectives.api import MaximizeObjective

objective = MaximizeObjective(
    w=1.0,
)
yield_feature = ContinuousOutput(key="Yield", objective=objective)
# create an output feature
output_features = Outputs(features=[yield_feature])


In [5]:
objective

MaximizeObjective(type='MaximizeObjective', w=1.0, bounds=(0, 1))

In [6]:
# we now have
print('input_features:', input_features)
print('output_features:', output_features)

input_features: type='Inputs' features=[ContinuousInput(type='ContinuousInput', key='Temperature', unit='°C', bounds=(30.0, 110.0), local_relative_bounds=None, stepsize=None), ContinuousInput(type='ContinuousInput', key='Catalyst Loading', unit='%', bounds=(0.5, 2.0), local_relative_bounds=None, stepsize=None), ContinuousInput(type='ContinuousInput', key='Residence Time', unit='minutes', bounds=(1.0, 10.0), local_relative_bounds=None, stepsize=None), CategoricalInput(type='CategoricalInput', key='Catalyst', categories=['[P1-L1]', '[P2-L1]', '[P1-L2]', '[P1-L3]', '[P1-L4]', '[P1-L5]', '[P1-L6]', '[P1-L7]'], allowed=[True, True, True, True, True, True, True, True])]
output_features: type='Outputs' features=[ContinuousOutput(type='ContinuousOutput', key='Yield', unit=None, objective=MaximizeObjective(type='MaximizeObjective', w=1.0, bounds=(0, 1)))]


In [7]:
# The domain is now the object that holds the entire optimization problem / problem definition.
domain = Domain(
    inputs = input_features,
    outputs = output_features,
)

In [8]:
# you can now have a pretty printout of your domain via
(domain.inputs+domain.outputs).get_reps_df()

,Type,Description
Catalyst Loading,ContinuousInput,"[0.5,2.0]"
Residence Time,ContinuousInput,"[1.0,10.0]"
Temperature,ContinuousInput,"[30.0,110.0]"
Catalyst,CategoricalInput,8 categories
Yield,ContinuousOutput,ContinuousOutputFeature


In [9]:
# and you can access your domain features via 
for feature_key in  domain.inputs.get_keys(): # this will get all the feature names and loop over them
    input_feature = domain.inputs.get_by_key(feature_key) # we can extract the individual feature object by asking for it by name
    print(feature_key, '|',  input_feature)

Catalyst Loading | [0.5,2.0]
Residence Time | [1.0,10.0]
Temperature | [30.0,110.0]
Catalyst | 8 categories


In [10]:
# as well as the output features as
# and you can access your domain features via 
for feature_key in  domain.outputs.get_keys(): # this will get all the feature names and loop over them
    output_feature = domain.outputs.get_by_key(feature_key) # we can extract the individual feature object by asking for it by name
    print(feature_key, ' | ',  output_feature.__repr__())

Yield  |  ContinuousOutput(type='ContinuousOutput', key='Yield', unit=None, objective=MaximizeObjective(type='MaximizeObjective', w=1.0, bounds=(0, 1)))


In [11]:
(domain.inputs+domain.outputs).get_reps_df()

,Type,Description
Catalyst Loading,ContinuousInput,"[0.5,2.0]"
Residence Time,ContinuousInput,"[1.0,10.0]"
Temperature,ContinuousInput,"[30.0,110.0]"
Catalyst,CategoricalInput,8 categories
Yield,ContinuousOutput,ContinuousOutputFeature
